In [1]:
from bs4 import BeautifulSoup
import requests
import json
from datetime import date
import pandas as pd

In [24]:
def get_html_data(urls,page_number):
    
    """Returns list of html data
    Each entry represents a page from the website and has multiple listings
    Takes URL, number of pages as input as well as the page url tag (ex: ?pagina=)
    TO DO: add functionality to return only 1 listing instead of the whole page of listings
    """
    
    url, url_with_page = urls
    
    full_list = []
    for i in range(1,page_number+1):
        if i == 1:
            req_result  = BeautifulSoup(requests.get(url).text,"lxml")
            offers_list = req_result.find_all('div',attrs={'class': 'box-anunt'})
            full_list.append(offers_list)
        else:
            url = url_with_page + str(i)
            req_result  = BeautifulSoup(requests.get(url).text,"lxml")
            offers_list = req_result.find_all('div',attrs={'class': 'box-anunt'})
            full_list.append(offers_list)
            
    print("Completed scrapping, returning list of entries")
    return full_list
    

In [13]:
def build_dict_hacky(scrapped_list,data_type):
    """ Build a dictionary based on the data points needed
    and the value we get back from our scrapping
    
    TO DO: add support for getting the subway proximity
    """
    
    out_dict = {}
    data_dict ={}
    
    for i in range(1,len(scrapped_list)):
        for j in range(1,len(scrapped_list[i])):
            try:
                record_id = data_type + "_" +str(i) + str(j) + "_" + str(date.today())
                location = scrapped_list[i][j].find('div',attrs={'class': 'localizare'}).text.strip()
                price = scrapped_list[i][j].find('span',attrs={'class': 'pret-mare'}).text.strip()
                features = scrapped_list[i][j].find('ul',attrs={'class': 'caracteristici'}).text.strip()
                
                out_dict[record_id] = dict(zip(["location","price","features"],[location, price, features]))
            except AttributeError:
                continue
                                           
    return out_dict  

In [119]:
def build_data_frame(scrapped_list,data_type):
    """ Build a data frame based on the data points needed
    and the value we get back from our scrapping
    
    TO DO: split the features into separate columns similar to location
    The splitting should be handled by a separate function...looks too messy the way it is
    """
    my_df = []
    for i in range(1,len(scrapped_list)):
        for j in range(1,len(scrapped_list[i])):
            try:
                town = scrapped_list[i][j].find('div',attrs={'class': 'localizare'}).find('p').text.strip().split()[0].strip(",")
                d = {
                "record_id" : town + "_" + data_type + "_" +str(i) + str(j) + "_" + str(date.today()),
                "town": town,
                "area": scrapped_list[i][j].find('div',attrs={'class': 'localizare'}).find('p').text.strip().split()[2],
                "price" : scrapped_list[i][j].find('span',attrs={'class': 'pret-mare'}).text.strip(),
                "features" : scrapped_list[i][j].find('ul',attrs={'class': 'caracteristici'}).text.strip()
                }
                my_df.append(d)
            except AttributeError:
                continue
    my_df = pd.DataFrame(my_df)
    
    return my_df

In [3]:
def url_build(city, prop_type, r_or_b):
    
    url_base = "https://www.imobiliare.ro/"
    
    if r_or_b == "sale":
        url = url_base + "vanzare-"
    else:
        url = url_base + "inchirieri-"
    
    if prop_type == "apts":
        url = url + "apartamente/"
    elif prop_type == "houses":
        url = url + "case-vile/"
    else:
        url = url + "garsoniere/"
    
    url = url + city
    multi_page = url + "?pagina="
    
    return url, multi_page 
    

In [128]:
cities = ["bucuresti","brasov","timisoara","cluj-napoca"]

In [129]:
re_data_df = pd.DataFrame()

In [130]:
for city in cities:
    df = build_data_frame(get_html_data(url_build(city, "apts", "sale"),100),"apts")
    re_data_df = re_data_df.append(df)

Completed scrapping, returning list of entries
Completed scrapping, returning list of entries
Completed scrapping, returning list of entries


IndexError: list index out of range

In [134]:
re_data_df.describe

<bound method NDFrame.describe of              area                                           features    price  \
0              1,  2 camere   53 mp utili   Parter/4   Decomandat...   63.900   
1         Domenii  3 camere   101 mp utili   Etaj 2/4   Semidecom...  150.000   
2        Militari  2 camere   54.04 mp utili   Etaj 6/6   Decoman...   41.400   
3              6,  2 camere   46 mp utili   Etaj 1/6   Decomandat...   39.000   
4              6,  2 camere   53 mp utili   Etaj 1/6   Decomandat...   43.000   
5              6,  2 camere   60 mp utili   Etaj 2   Decomandat  ...   51.000   
6              6,  3 camere   74.69 mp utili   Etaj 3/14   Decoma...   66.733   
7        Militari  2 camere   59.93 mp utili   Etaj 6/6   Decoman...   45.900   
8              6,  2 camere   52 mp utili   Etaj 2/11   Decomanda...   63.500   
9              6,  o cameră   39 mp utili   Etaj 2/6   Decomandat...   36.000   
10       Aviatiei  3 camere   70.9 mp utili   Etaj 3/7   Decomand...  103.1

In [121]:
cluj_apts_df.head()

,area,features,price,record_id,town
0,Manastur,2 camere 51 mp utili Etaj 4/4 Semidecoma...,67.000,Cluj-Napoca_apts_11_2018-10-11,Cluj-Napoca
1,Manastur,2 camere 52.1 mp utili Etaj 2/9 Decomand...,65.714,Cluj-Napoca_apts_12_2018-10-11,Cluj-Napoca
2,Calea,2 camere 53 mp utili Etaj 1 Decomandat ...,60.000,Cluj-Napoca_apts_13_2018-10-11,Cluj-Napoca
3,Andrei,2 camere 72 mp utili Parter Semidecomandat,123.000,Cluj-Napoca_apts_14_2018-10-11,Cluj-Napoca
4,Semicentral,2 camere 47 mp utili Etaj 1/1 Semidecoma...,63.000,Cluj-Napoca_apts_16_2018-10-11,Cluj-Napoca


In [72]:
cluj_ap_df["price"] = pd.to_numeric(cluj_ap_df["price"], errors='coerce')
cluj_ap_df[['rooms','size','floor','style',"other feature"]] = cluj_ap_df.features.str.split('  ', expand = True)

AttributeError: 'DataFrame' object has no attribute 'features'

all below this line are legacy 

In [ ]:
buc_houses_df = build_data_frame(get_html_data(url_build("bucuresti", "houses", "sale"),100),"houses")
buc_gars_df = build_data_frame(get_html_data(url_build("bucuresti", "onebdr", "sale"),100),"onebdr")

In [44]:
def correct_prices(df_name,col,mult):
    """Takes a series, df column and changes it to numeric
    and multiplies by mult
    """
    
    df_name[col] = pd.to_numeric(df_name[col], errors='coerce')
    df_name[col].loc[:,col]*=mult
                                         
    return df_name[col]

In [46]:
buc_gars_df["price"] = pd.to_numeric(buc_gars_df["price"], errors='coerce')
buc_houses_df["price"] = pd.to_numeric(buc_houses_df["price"], errors='coerce')
buc_ap_df["price"] = pd.to_numeric(buc_ap_df["price"], errors='coerce')

In [47]:
buc_gars_df.loc[:,'price'] *= 1000
buc_houses_df.loc[:,'price'] *= 1000
buc_ap_df.loc[:,'price'] *= 1000

In [60]:
buc_ap_df[['rooms','size','floor','style',"other feature"]] = buc_ap_df.features.str.split('  ', expand = True)
buc_houses_df[['rooms','size','floor','style',"other feature"]] = buc_houses_df.features.str.split('  ', expand = True)
buc_gars_df[['rooms','size','floor','style',"other feature"]] = buc_gars_df.features.str.split('  ', expand = True)

ValueError: Columns must be same length as key

In [69]:
buc_gars_df["features"][0].split('  ')

['o cameră', ' 32.1 mp utili', ' Etaj 3', ' Decomandat', ' Bloc nou']

In [55]:
buc_ap_df

,area,dist_to_subway,features,price,record_id,town,rooms,size,floor,style,other feature
0,"6,",1720,2 camere 46 mp utili Etaj 1/6 Decomandat...,39000.0,apts_11_2018-10-10,"Bucuresti,",2 camere,46 mp utili,Etaj 1/6,Decomandat,Bloc nou
1,"6,",1720,2 camere 53 mp utili Etaj 1/6 Decomandat...,43000.0,apts_12_2018-10-10,"Bucuresti,",2 camere,53 mp utili,Etaj 1/6,Decomandat,Bloc nou
2,"6,",1390,2 camere 60 mp utili Etaj 2 Decomandat ...,51000.0,apts_13_2018-10-10,"Bucuresti,",2 camere,60 mp utili,Etaj 2,Decomandat,Bloc nou
3,"6,",830,3 camere 74.69 mp utili Etaj 3/14 Decoma...,66733.0,apts_14_2018-10-10,"Bucuresti,",3 camere,74.69 mp utili,Etaj 3/14,Decomandat,Bloc nou
4,Militari,1940,2 camere 59.93 mp utili Etaj 6/6 Decoman...,45900.0,apts_16_2018-10-10,"Bucuresti,",2 camere,59.93 mp utili,Etaj 6/6,Decomandat,Bloc nou
5,"6,",950,o cameră 39 mp utili Etaj 2/6 Decomandat...,36000.0,apts_17_2018-10-10,"Bucuresti,",o cameră,39 mp utili,Etaj 2/6,Decomandat,Bloc nou
6,"6,",190,2 camere 52 mp utili Etaj 2/11 Decomanda...,63500.0,apts_18_2018-10-10,"Bucuresti,",2 camere,52 mp utili,Etaj 2/11,Decomandat,Bloc nou
7,Aviatiei,1440,3 camere 70.9 mp utili Etaj 3/7 Decomand...,103162.0,apts_19_2018-10-10,"Bucuresti,",3 camere,70.9 mp utili,Etaj 3/7,Decomandat,Bloc nou
8,"6,",940,4 camere 100 mp utili Etaj 2/3 Decomanda...,76900.0,apts_110_2018-10-10,"Bucuresti,",4 camere,100 mp utili,Etaj 2/3,Decomandat,Bloc nou
9,Rahova,30,2 camere 75 mp utili Etaj 1 Decomandat,49000.0,apts_111_2018-10-10,"Bucuresti,",2 camere,75 mp utili,Etaj 1,Decomandat,None
